### 폐암 환자 데이터셋을 활용한 이진 분류

- 17개의 특징으로 구성
- 라벨은 생존(1),사망(0) 클래스로 구성

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [3]:
data=pd.read_csv("./data/ThoraricSurgery.csv",header=None)
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,293,1,3.80,2.80,0,0,0,0,0,0,12,0,0,0,1,0,62,0
1,1,2,2.88,2.16,1,0,0,0,1,1,14,0,0,0,1,0,60,0
2,8,2,3.19,2.50,1,0,0,0,1,0,11,0,0,1,1,0,66,1
3,14,2,3.98,3.06,2,0,0,0,1,1,14,0,0,0,1,0,80,1
4,17,2,2.21,1.88,0,0,1,0,0,0,12,0,0,0,1,0,56,0


In [4]:
#결측치
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 470 entries, 0 to 469
Data columns (total 18 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       470 non-null    int64  
 1   1       470 non-null    int64  
 2   2       470 non-null    float64
 3   3       470 non-null    float64
 4   4       470 non-null    int64  
 5   5       470 non-null    int64  
 6   6       470 non-null    int64  
 7   7       470 non-null    int64  
 8   8       470 non-null    int64  
 9   9       470 non-null    int64  
 10  10      470 non-null    int64  
 11  11      470 non-null    int64  
 12  12      470 non-null    int64  
 13  13      470 non-null    int64  
 14  14      470 non-null    int64  
 15  15      470 non-null    int64  
 16  16      470 non-null    int64  
 17  17      470 non-null    int64  
dtypes: float64(2), int64(16)
memory usage: 66.2 KB


In [5]:
#이상치
data.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
count,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000
mean,235.500000,3.095745,3.281638,4.568702,0.780851,0.065957,0.144681,0.065957,0.687234,0.165957,11.736170,0.074468,0.004255,0.017021,0.821277,0.004255,62.534043,0.148936
std,135.821574,0.722309,0.871395,11.767857,0.535375,0.248472,0.352154,0.248472,0.464114,0.372439,0.702243,0.262811,0.065163,0.129488,0.383529,0.065163,8.706902,0.356405
min,1.000000,1.000000,1.440000,0.960000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11.000000,0.000000,0.000000,0.000000,0.000000,0.000000,21.000000,0.000000
25%,118.250000,3.000000,2.600000,1.960000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11.000000,0.000000,0.000000,0.000000,1.000000,0.000000,57.000000,0.000000
50%,235.500000,3.000000,3.160000,2.400000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,12.000000,0.000000,0.000000,0.000000,1.000000,0.000000,62.000000,0.000000
75%,352.750000,3.000000,3.807500,3.080000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,12.000000,0.000000,0.000000,0.000000,1.000000,0.000000,69.000000,0.000000
max,470.000000,8.000000,6.300000,86.300000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,14.000000,1.000000,1.000000,1.000000,1.000000,1.000000,87.000000,1.000000


In [6]:
#상관관계
data.corr()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,1.000000,-0.007878,-0.034985,0.029563,-0.021449,0.052724,0.027238,-0.061885,0.008845,0.027271,0.047079,-0.009229,-0.016382,-0.015761,-0.038681,-0.042400,-0.005826,-0.074924
1,-0.007878,1.000000,0.121263,0.058780,-0.055900,0.047900,-0.062958,-0.047142,-0.075850,-0.019562,-0.135050,0.029753,-0.008675,-0.017461,-0.107427,-0.008675,0.076271,0.060444
2,-0.034985,0.121263,1.000000,0.032975,-0.091094,0.019786,-0.095827,0.055829,-0.052770,-0.100242,0.034088,-0.115145,-0.009135,-0.035584,-0.012009,-0.060578,-0.290178,-0.046374
3,0.029563,0.058780,0.032975,1.000000,-0.143155,0.161615,0.102979,0.260073,-0.099914,-0.086103,0.015504,-0.022251,-0.013617,-0.025088,-0.100853,-0.016509,-0.115900,-0.042841
4,-0.021449,-0.055900,-0.091094,-0.143155,1.000000,0.092863,0.123296,0.092863,0.684647,0.418042,0.089751,0.025310,0.026788,0.023166,0.172289,-0.034330,0.214528,0.093200
5,0.052724,0.047900,0.019786,0.161615,0.092863,1.000000,0.256225,0.067529,-0.024115,-0.072455,0.099942,0.022578,-0.017372,-0.034968,-0.077406,-0.017372,0.044789,0.057375
6,0.027238,-0.062958,-0.095827,0.102979,0.123296,0.256225,1.000000,0.134386,0.081772,0.060393,0.059840,-0.001471,-0.026886,0.086156,-0.044942,-0.026886,0.086705,0.065785
7,-0.061885,-0.047142,0.055829,0.260073,0.092863,0.067529,0.134386,1.000000,0.049843,-0.072455,0.075502,-0.042725,-0.017372,0.097572,-0.077406,-0.017372,-0.015331,0.105530
8,0.008845,-0.075850,-0.052770,-0.099914,0.684647,-0.024115,0.081772,0.049843,1.000000,0.202245,0.145345,0.016551,0.044101,0.017815,0.200373,-0.026401,0.149589,0.088860
9,0.027271,-0.019562,-0.100242,-0.086103,0.418042,-0.072455,0.060393,-0.072455,0.202245,1.000000,-0.036044,0.069522,0.058695,0.029726,0.118527,-0.029161,0.208003,0.086467


In [7]:
#X,y로 분리
X=data.iloc[:,:-1]
y=data.iloc[:,-1]
X.shape,y.shape

((470, 17), (470,))

신경망 설계

In [8]:
#w,b를 초기화하기 위한 seed를 설정
seed=7
np.random.seed(seed)
tf.random.set_seed(seed)

In [9]:
#신경망 설계
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [11]:
#새로운 신경망 생성
model=Sequential()
#신경망에 입력층을 추가
#Dense 첫 파라미터는 가급적 2의 n승을 맞춰서 해줌
model.add(Dense(34,input_dim=17,activation="relu"))
#신경망에 출력층을 추가
#회귀 : linear(기본값이라 안써도 됨)
#이진분류 : 1(sigmoid),2(softmax,y를 원핫인코딩한 경우)
#다중분류 : softmax
model.add(Dense(1,activation="sigmoid"))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 34)                612       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 35        
Total params: 647
Trainable params: 647
Non-trainable params: 0
_________________________________________________________________


In [12]:
#컴파일
model.compile(loss="binary_crossentropy",
             optimizer="adam",
             metrics=["acc"])

In [13]:
#학습
model.fit(X,y,epochs=30,batch_size=10)

Epoch 1/30
47/47 [==============================] - 0s 520us/step - loss: 7.8274 - acc: 0.7680
Epoch 2/30
47/47 [==============================] - 0s 499us/step - loss: 1.4967 - acc: 0.7098
Epoch 3/30
47/47 [==============================] - 0s 499us/step - loss: 0.5225 - acc: 0.8515
Epoch 4/30
47/47 [==============================] - 0s 434us/step - loss: 0.4477 - acc: 0.8657
Epoch 5/30
47/47 [==============================] - 0s 585us/step - loss: 0.4878 - acc: 0.8531
Epoch 6/30
47/47 [==============================] - 0s 564us/step - loss: 0.4375 - acc: 0.8556
Epoch 7/30
47/47 [==============================] - 0s 477us/step - loss: 0.5377 - acc: 0.7890
Epoch 8/30
47/47 [==============================] - 0s 455us/step - loss: 0.5734 - acc: 0.7809
Epoch 9/30
47/47 [==============================] - 0s 477us/step - loss: 0.4932 - acc: 0.8513
Epoch 10/30
47/47 [==============================] - 0s 5ms/step - loss: 0.4204 - acc: 0.8553
Epoch 11/30
47/47 [==============================] 

In [14]:
model.evaluate(X,y)

15/15 [==============================] - 0s 500us/step - loss: 0.3895 - acc: 0.8511


[0.3894892930984497, 0.8510638475418091]

In [15]:
model1=Sequential()

#입력층
model1.add(Dense(34,input_dim=17,activation="relu"))

#은닉층 추가
model1.add(Dense(17,activation="relu"))

#출력층
model1.add(Dense(1,activation="sigmoid"))
model1.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 34)                612       
_________________________________________________________________
dense_5 (Dense)              (None, 17)                595       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 18        
Total params: 1,225
Trainable params: 1,225
Non-trainable params: 0
_________________________________________________________________


In [16]:
model1.compile(loss="binary_crossentropy",
              optimizer="adam",
              metrics=["acc"])

In [17]:
model1.fit(X,y,epochs=30,batch_size=10)

Epoch 1/30
47/47 [==============================] - 0s 521us/step - loss: 2.2515 - acc: 0.7848
Epoch 2/30
47/47 [==============================] - 0s 521us/step - loss: 0.5954 - acc: 0.8218
Epoch 3/30
47/47 [==============================] - 0s 585us/step - loss: 0.4894 - acc: 0.8595
Epoch 4/30
47/47 [==============================] - 0s 586us/step - loss: 0.4943 - acc: 0.8479
Epoch 5/30
47/47 [==============================] - 0s 498us/step - loss: 0.4742 - acc: 0.8427
Epoch 6/30
47/47 [==============================] - 0s 499us/step - loss: 0.4254 - acc: 0.8532
Epoch 7/30
47/47 [==============================] - 0s 499us/step - loss: 0.6496 - acc: 0.7685
Epoch 8/30
47/47 [==============================] - 0s 455us/step - loss: 0.5742 - acc: 0.7669
Epoch 9/30
47/47 [==============================] - 0s 499us/step - loss: 0.4931 - acc: 0.8568
Epoch 10/30
47/47 [==============================] - 0s 434us/step - loss: 0.4159 - acc: 0.8561
Epoch 11/30
47/47 [==============================

In [18]:
model1.evaluate(X,y)

15/15 [==============================] - 0s 498us/step - loss: 0.3895 - acc: 0.8511


[0.38945600390434265, 0.8510638475418091]

In [19]:
model2=Sequential()

#입력층
model2.add(Dense(128,input_dim=17,activation="relu"))

#은닉층 추가
model2.add(Dense(256,activation="relu"))

#출력층
model2.add(Dense(1,activation="sigmoid"))
model2.summary()
model2.compile(loss="binary_crossentropy",
              optimizer="adam",
              metrics=["acc"])

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 128)               2304      
_________________________________________________________________
dense_8 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 257       
Total params: 35,585
Trainable params: 35,585
Non-trainable params: 0
_________________________________________________________________


In [20]:
model2.fit(X,y,epochs=30,batch_size=10)

Epoch 1/30
47/47 [==============================] - 0s 607us/step - loss: 3.5890 - acc: 0.7395
Epoch 2/30
47/47 [==============================] - 0s 498us/step - loss: 0.6812 - acc: 0.8013
Epoch 3/30
47/47 [==============================] - 0s 629us/step - loss: 0.7147 - acc: 0.7835
Epoch 4/30
47/47 [==============================] - 0s 564us/step - loss: 0.6052 - acc: 0.8009
Epoch 5/30
47/47 [==============================] - 0s 672us/step - loss: 0.5083 - acc: 0.8487
Epoch 6/30
47/47 [==============================] - 0s 759us/step - loss: 0.5245 - acc: 0.8405
Epoch 7/30
47/47 [==============================] - 0s 520us/step - loss: 0.6822 - acc: 0.7796
Epoch 8/30
47/47 [==============================] - 0s 629us/step - loss: 0.6179 - acc: 0.7696
Epoch 9/30
47/47 [==============================] - 0s 564us/step - loss: 0.7666 - acc: 0.7734
Epoch 10/30
47/47 [==============================] - 0s 542us/step - loss: 0.5186 - acc: 0.8413
Epoch 11/30
47/47 [==============================

In [23]:
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout

In [74]:
model3=Sequential()

#입력층
model3.add(Dense(17,input_dim=17,activation="relu"))
#은닉층 추가
model3.add(Dense(34,activation="relu"))
model3.add(Dense(68,activation="relu"))
model3.add(Dense(34,activation="relu"))
model3.add(Dense(17,activation="relu"))
#출력층
model3.add(Dense(1,activation="sigmoid"))
model3.summary()
model3.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["acc"])

Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_109 (Dense)            (None, 17)                306       
_________________________________________________________________
dense_110 (Dense)            (None, 34)                612       
_________________________________________________________________
dense_111 (Dense)            (None, 68)                2380      
_________________________________________________________________
dense_112 (Dense)            (None, 34)                2346      
_________________________________________________________________
dense_113 (Dense)            (None, 17)                595       
_________________________________________________________________
dense_114 (Dense)            (None, 1)                 18        
Total params: 6,257
Trainable params: 6,257
Non-trainable params: 0
___________________________________________________

In [75]:
model3.fit(X,y,epochs=100,batch_size=300)

Epoch 1/100
2/2 [==============================] - 1s 3ms/step - loss: 2.5725 - acc: 0.3336
Epoch 2/100
2/2 [==============================] - 0s 2ms/step - loss: 0.7870 - acc: 0.8398
Epoch 3/100
2/2 [==============================] - 0s 2ms/step - loss: 0.4467 - acc: 0.8534
Epoch 4/100
2/2 [==============================] - 0s 2ms/step - loss: 0.4494 - acc: 0.8552
Epoch 5/100
2/2 [==============================] - 0s 997us/step - loss: 0.4562 - acc: 0.8540
Epoch 6/100
2/2 [==============================] - 0s 997us/step - loss: 0.4392 - acc: 0.8540
Epoch 7/100
2/2 [==============================] - 0s 2ms/step - loss: 0.4590 - acc: 0.8452
Epoch 8/100
2/2 [==============================] - 0s 2ms/step - loss: 0.6818 - acc: 0.8518
Epoch 9/100
2/2 [==============================] - 0s 998us/step - loss: 0.4814 - acc: 0.8510
Epoch 10/100
2/2 [==============================] - 0s 2ms/step - loss: 0.4177 - acc: 0.8566
Epoch 11/100
2/2 [==============================] - 0s 2ms/step - loss: 0

In [76]:
model3.evaluate(X,y)

15/15 [==============================] - 0s 570us/step - loss: 0.4044 - acc: 0.8532


[0.4043582081794739, 0.8531914949417114]

In [73]:
#실제값하고 예측값을 비교
pred=model1.predict(X)

for i in range(10):
    print("실제값 : {}\t 예측값 {}".format(y[i], pred[i]))

실제값 : 0	 예측값 [0.08831817]
실제값 : 0	 예측값 [0.10814926]
실제값 : 1	 예측값 [0.48992684]
실제값 : 1	 예측값 [0.33813292]
실제값 : 0	 예측값 [0.10479727]
실제값 : 0	 예측값 [0.09001258]
실제값 : 0	 예측값 [0.1697509]
실제값 : 1	 예측값 [0.20049879]
실제값 : 0	 예측값 [0.19138545]
실제값 : 0	 예측값 [0.14549899]


이진분류를 y를 원핫인코딩해서 실행

In [77]:
y.shape

(470,)

In [78]:
y_en=pd.get_dummies(y)
y_en.shape

(470, 2)

In [79]:
y_en

,0,1
0,1,0
1,1,0
2,0,1
3,0,1
4,1,0
...,...,...
465,1,0
466,1,0
467,1,0
468,0,1


In [89]:
model4=Sequential()

#입력층
model4.add(Dense(34,input_dim=17,activation="relu"))
#은닉층 추가
model4.add(Dense(51,activation="relu"))
model4.add(Dense(68,activation="relu"))
model4.add(Dense(34,activation="relu"))
model4.add(Dense(17,activation="relu"))
#출력층
model4.add(Dense(2,activation="softmax"))
model4.summary()
model4.compile(loss="binary_crossentropy",
              optimizer="adam",
              metrics=["acc"])

Model: "sequential_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_133 (Dense)            (None, 34)                612       
_________________________________________________________________
dense_134 (Dense)            (None, 51)                1785      
_________________________________________________________________
dense_135 (Dense)            (None, 68)                3536      
_________________________________________________________________
dense_136 (Dense)            (None, 34)                2346      
_________________________________________________________________
dense_137 (Dense)            (None, 17)                595       
_________________________________________________________________
dense_138 (Dense)            (None, 2)                 36        
Total params: 8,910
Trainable params: 8,910
Non-trainable params: 0
___________________________________________________

In [90]:
model4.fit(X,y_en,epochs=100,batch_size=10)

Epoch 1/100
47/47 [==============================] - 1s 564us/step - loss: 0.8248 - acc: 0.8280
Epoch 2/100
47/47 [==============================] - 0s 607us/step - loss: 0.4510 - acc: 0.8560
Epoch 3/100
47/47 [==============================] - 0s 651us/step - loss: 0.3962 - acc: 0.8704
Epoch 4/100
47/47 [==============================] - 0s 628us/step - loss: 0.4139 - acc: 0.8682
Epoch 5/100
47/47 [==============================] - 0s 607us/step - loss: 0.4264 - acc: 0.8565
Epoch 6/100
47/47 [==============================] - 0s 585us/step - loss: 0.4166 - acc: 0.8560
Epoch 7/100
47/47 [==============================] - 0s 564us/step - loss: 0.4965 - acc: 0.8289
Epoch 8/100
47/47 [==============================] - 0s 564us/step - loss: 0.4651 - acc: 0.8368
Epoch 9/100
47/47 [==============================] - 0s 564us/step - loss: 0.4348 - acc: 0.8499
Epoch 10/100
47/47 [==============================] - 0s 564us/step - loss: 0.3958 - acc: 0.8674
Epoch 11/100
47/47 [===================

In [91]:
model4.evaluate(X,y_en)

15/15 [==============================] - 0s 499us/step - loss: 0.3993 - acc: 0.8532


[0.3993336856365204, 0.8531914949417114]

In [96]:
pred=model4.predict(X)

In [97]:
y_en["pred_0"]=pred[:,0]
y_en["pred_1"]=pred[:,1]

y_en

,0,1,pred_0,pred_1
0,1,0,0.964063,0.035937
1,1,0,0.421051,0.578949
2,0,1,0.387686,0.612314
3,0,1,0.397467,0.602533
4,1,0,0.984820,0.015180
...,...,...,...,...
465,1,0,0.978221,0.021779
466,1,0,0.978946,0.021054
467,1,0,0.985210,0.014790
468,0,1,0.875227,0.124773


### 보스턴 집값 데이터를 이용한 회귀 학습

In [98]:
from sklearn.datasets import load_boston

In [99]:
from tensorflow.keras.datasets import boston_housing

In [100]:
(X_train,y_train),(X_test,y_test)=boston_housing.load_data()

57344/57026 [==============================] - 0s 0us/step


In [101]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((404, 13), (102, 13), (404,), (102,))

In [118]:
model_boston=Sequential()
#입력층
model_boston.add(Dense(128,input_dim=13,activation="relu"))
#은닉층
model_boston.add(Dense(52,activation="relu"))
model_boston.add(Dense(26,activation="relu"))
model_boston.add(Dense(13,activation="relu"))
#출력층
model_boston.add(Dense(1,activation="linear"))
model_boston.summary()

Model: "sequential_34"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_157 (Dense)            (None, 128)               1792      
_________________________________________________________________
dense_158 (Dense)            (None, 52)                6708      
_________________________________________________________________
dense_159 (Dense)            (None, 26)                1378      
_________________________________________________________________
dense_160 (Dense)            (None, 13)                351       
_________________________________________________________________
dense_161 (Dense)            (None, 1)                 14        
Total params: 10,243
Trainable params: 10,243
Non-trainable params: 0
_________________________________________________________________


In [119]:
#컴파일
#회귀 모델이므로 mse 또는 mean squared error
#metrics는 안써도 됨 -loss만 평가
model_boston.compile(loss="mse",optimizer="adam")

In [130]:
#verbose=0:실행과정을 보여주지 않게 설정
model_boston.fit(X_train,y_train,epochs=15000,batch_size=10,verbose=0)

In [131]:
model_boston.evaluate(X_train,y_train)

13/13 [==============================] - 0s 499us/step - loss: 0.0369


0.03685653209686279

### BMI 데이터셋을 이용한 다중분류 학습

In [3]:
bmi=pd.read_csv("./data/bmi.csv")
bmi.head()

,height,weight,label
0,140,45,normal
1,145,72,fat
2,150,61,fat
3,137,56,fat
4,192,48,thin


In [4]:
bmi['label'].shape

(20000,)

In [133]:
bmi['label'].unique()

array(['normal', 'fat', 'thin'], dtype=object)

In [134]:
bmi['label'].value_counts()

fat       7712
thin      6338
normal    5950
Name: label, dtype: int64

In [138]:
from sklearn.model_selection import train_test_split

In [139]:
X=bmi.iloc[:,:2]
y=bmi.iloc[:,2]

In [140]:
X_train,X_test,y_train,y_test=train_test_split(X,y)

In [141]:
y_train=pd.get_dummies(y_train)
y_test=pd.get_dummies(y_test)

In [154]:
#딥러닝을 이용해서 다중분류
model_bmi=Sequential()
model_bmi.add(Dense(128,input_dim=2,activation="relu"))
model_bmi.add(Dense(256,activation="relu"))
model_bmi.add(Dense(128,activation="relu"))
model_bmi.add(Dense(256,activation="relu"))
model_bmi.add(Dense(128,activation="relu"))
model_bmi.add(Dense(3,activation="softmax"))
model_bmi.summary()

Model: "sequential_39"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_180 (Dense)            (None, 128)               384       
_________________________________________________________________
dense_181 (Dense)            (None, 256)               33024     
_________________________________________________________________
dense_182 (Dense)            (None, 128)               32896     
_________________________________________________________________
dense_183 (Dense)            (None, 256)               33024     
_________________________________________________________________
dense_184 (Dense)            (None, 128)               32896     
_________________________________________________________________
dense_185 (Dense)            (None, 3)                 387       
Total params: 132,611
Trainable params: 132,611
Non-trainable params: 0
_______________________________________________

In [155]:
model_bmi.compile(loss="categorical_crossentropy",
                 optimizer="adam",
                 metrics=["acc"])

In [156]:
model_bmi.fit(X_train,y_train,epochs=100,batch_size=10)

Epoch 1/100
1500/1500 [==============================] - 2s 875us/step - loss: 0.6793 - acc: 0.7111
Epoch 2/100
1500/1500 [==============================] - 1s 963us/step - loss: 0.3806 - acc: 0.8238
Epoch 3/100
1500/1500 [==============================] - 1s 897us/step - loss: 0.1888 - acc: 0.9245 1s - 
Epoch 4/100
1500/1500 [==============================] - 1s 905us/step - loss: 0.1405 - acc: 0.9394
Epoch 5/100
1500/1500 [==============================] - 1s 955us/step - loss: 0.1077 - acc: 0.9561
Epoch 6/100
1500/1500 [==============================] - 2s 1ms/step - loss: 0.0941 - acc: 0.9608
Epoch 7/100
1500/1500 [==============================] - 1s 896us/step - loss: 0.0917 - acc: 0.9613
Epoch 8/100
1500/1500 [==============================] - 1s 932us/step - loss: 0.0957 - acc: 0.9619
Epoch 9/100
1500/1500 [==============================] - 1s 940us/step - loss: 0.0880 - acc: 0.9633
Epoch 10/100
1500/1500 [==============================] - 1s 902us/step - loss: 0.0774 - acc: 0.

In [157]:
model_bmi.evaluate(X_test,y_test)

157/157 [==============================] - 0s 620us/step - loss: 0.0565 - acc: 0.9712


[0.056547705084085464, 0.9711999893188477]